In [1]:
!echo ${TVM_HOME}

/home/silver/workbench/tvm-unity


In [2]:
import tvm
from tvm import relay, relax
print(tvm.__version__)
import numpy as np

import torch

0.14.dev0


## Import Tiny llamas

In [3]:
from model import *
device = "cpu"
ckpt_path = "tinyllamas/stories15M.pt"
checkpoint = torch.load(ckpt_path, map_location=device)
checkpoint_model_args = checkpoint["model_args"]

model_args = dict()

for k in ["dim", "n_layers", "n_heads", "n_kv_heads", "vocab_size", "multiple_of", "max_seq_len"]:
    model_args[k] = checkpoint_model_args[k]
# create the model
gptconf = ModelArgs(**model_args)
model = Transformer(gptconf)
state_dict = checkpoint["model"]
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = "_orig_mod."
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
model.load_state_dict(state_dict)
iter_num = checkpoint["iter_num"]
best_val_loss = checkpoint["best_val_loss"]
model.to(device)

model.eval()

Transformer(
  (tok_embeddings): Embedding(32000, 288)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, ou

## Test model

In [4]:
from tokenizer import Tokenizer

vocab_size = gptconf.vocab_size
tokenizer_model = "tokenizer.model"
enc = Tokenizer(tokenizer_model=tokenizer_model)

start = "Jerry is a little boy"

start_ids = enc.encode(start, bos=True, eos=False)
input_data = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
print(input_data)
# scripted_model = torch.jit.trace(model, input_data).eval()

tensor([[    1, 23052,   338,   263,  2217,  8023]])


## Import to Relay

In [5]:
input_name = "input0"
shape_list = [(input_name, input_data.shape)]


# Relay
# mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

# Relax
from tvm.relax.frontend.torch import from_fx
from torch import fx

symbolic_model = fx.symbolic_trace(model)

mod, params = from_fx(symbolic_model, shape_list)


# TODO: Fuck, no way to convert it! TVM has no support for it.

TypeError: slice indices must be integers or None or have an __index__ method